# iNaturalist status updates by state

Using the file produced in the collate-status-taxa.ipynb: `inat-aust-status-taxa.csv`, generate lists to update iNaturalist statuses

**Next steps:**
State by state establish the changes that need to be made:
    a. new - any new species that appear in the state lists but do not have a status in inaturalist (new template)
    b. updates - any changes to information which was added by us previously (user_id = 708886) (update template, action='UPDATE')
    c. removals - any statuses which were added by us previously (user_id = 708886) list which are incorrect (update template, action='REMOVE')
    d. flags - are there any statuses by other users that need to be flagged?

## Prep - common to all states
1. Read in the inaturalist statuses & filter out this state
2. Read in the inaturalist taxa list
3. Read in the state sensitive list
4. Attempt to match the state statuses to an IUCN equivalent


### 1. iNaturalist statuses

In [64]:
import pandas as pd
import sys
import os
projectdir = "/Users/oco115/PycharmProjects/authoritative-lists/" # basedir for this gh project
sys.path.append(os.path.abspath(projectdir + "source-code/includes"))
import list_functions as lf

sourcedir = projectdir + "source-data/inaturalist-statuses/"
listdir = projectdir + "current-lists/"


# read in the statuses
taxastatus = pd.read_csv(sourcedir + "inat-aust-status-taxa.csv", encoding='UTF-8',na_filter=False,dtype=str) ## Read inaturalist conservation statuses file
# taxastatus.head(3)
taxastatus

,id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
0,166449,38493,1138587,7830,,Flora and Fauna Guarantee Act 1988,CR,,,obscured,...,Eulamprus,kosciuskoi,,2021-03-01T10:35:01Z,Eulamprus kosciuskoi,species,http://reptile-database.reptarium.cz/search.ph...,,,
1,234788,918383,702203,9994,,Atlas of Living Australia,NT,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,
2,234789,918383,702203,7308,,Atlas of Living Australia,LC,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,
3,166416,1033183,3669610,6825,,NSW Office of Environment & Heritage,EN,https://www.environment.nsw.gov.au/threateneds...,,obscured,...,Eidothea,hardeniana,,2021-02-22T07:21:17Z,Eidothea hardeniana,species,,,,
4,180721,1247288,222137,6825,,NSW Threatened Species Scientific Committee,vu,https://www.environment.nsw.gov.au/topics/anim...,,obscured,...,Pomaderris,bodalla,,2021-08-27T06:18:35Z,Pomaderris bodalla,species,https://eol.org/pages/49432063,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3448,165697,1182117,3669610,73684,,Australian Government,CR,http://www.environment.gov.au/biodiversity/thr...,,,...,Achyranthes,margaretarum,,2021-02-12T19:39:39Z,Achyranthes margaretarum,species,http://plantsoftheworldonline.org/taxon/urn:ls...,,,
3449,130966,508985,,,,,critically endangered,http://environment.gov.au/cgi-bin/sprat/public...,,obscured,...,Lichenostomus,melanops,cassidix,2022-06-11T03:51:49Z,Lichenostomus melanops cassidix,subspecies,http://www.birds.cornell.edu/clementschecklist...,,,
3450,161226,50744,516268,,,DAWE Species Profile and Threats Database,CR,https://www.environment.gov.au/cgi-bin/sprat/p...,,obscured,...,,,,,Stiphodon allen,,,Opal Cling-Goby,False,[]
3451,162783,924263,764897,,,"Department of Biodiversity, Conservation and A...",EN,https://www.dpaw.wa.gov.au/images/documents/pl...,,obscured,...,Reedia,spathacea,,2020-09-27T12:27:42Z,Reedia spathacea,species,,,,


In [65]:
def filter_state_statuses(stateregex: str, urlregex: str):
    authoritydf = taxastatus['authority'].drop_duplicates().sort_values()
    authoritydf = authoritydf[pd.Series(authoritydf).str.contains(stateregex)]
    urldf = taxastatus['url'].drop_duplicates().sort_values()
    urldf = urldf[pd.Series(urldf).str.contains(urlregex)]
    placedisplaydf = taxastatus['place_display_name'].drop_duplicates().sort_values()
    placedisplaydf = placedisplaydf[pd.Series(placedisplaydf).str.contains(stateregex)]
    placedf = taxastatus['place_name'].drop_duplicates().sort_values()
    placedf = placedf[pd.Series(placedf).str.contains(stateregex)]
    # concat all and remove duplicates
    statedf = pd.concat([taxastatus.apply(lambda row: row[taxastatus['place_display_name'].isin(placedisplaydf)]),
                         taxastatus.apply(lambda row: row[taxastatus['place_name'].isin(placedf)]),
                         taxastatus.apply(lambda row: row[taxastatus['url'].isin(urldf)]),
                         taxastatus.apply(
                             lambda row: row[taxastatus['authority'].isin(authoritydf)])]).drop_duplicates()
    return statedf.sort_values(['taxon_id', 'user_id'])
inatstatuses = filter_state_statuses("NSW|New South Wales", ".nsw.")
inatstatuses.rename(columns={'id':'status_id','id_y':'taxon_id_y'},inplace=True)
inatstatuses

,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
3,166416,1033183,3669610,6825,,NSW Office of Environment & Heritage,EN,https://www.environment.nsw.gov.au/threateneds...,,obscured,...,Eidothea,hardeniana,,2021-02-22T07:21:17Z,Eidothea hardeniana,species,,,,
2656,165059,1054498,58320,6825,,NSW Office of Environment & Heritage,Vulnerable,https://www.environment.nsw.gov.au/threatenedS...,,obscured,...,Prostanthera,cryptandroides,cryptandroides,2020-12-17T03:53:45Z,Prostanthera cryptandroides cryptandroides,subspecies,,,,
477,264941,1061113,3669610,6825,,New South Wales Biodiversity Conservation Act ...,EX,https://bie.ala.org.au/species/https://id.biod...,Presumed Extinct,open,...,Leuzea,australis,,2022-06-11T11:38:00Z,Leuzea australis,species,,,,
371,180988,1070573,3669610,6825,,NSW Office of Environment & Heritage,EN,https://www.environment.nsw.gov.au/threateneds...,,,...,Rotala,tripartita,,2021-09-21T05:48:55Z,Rotala tripartita,species,https://eol.org/pages/49427430,,,
2514,160581,1076814,990532,6825,,NSW Office of Environment & Heritage,Vulnerable,https://www.environment.nsw.gov.au/threateneds...,,obscured,...,Pterostylis,nigricans,,2022-07-12T14:19:22Z,Pterostylis nigricans,species,http://www.catalogueoflife.org/annual-checklis...,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,152247,906670,708886,6825,16650,NSW Office of Environment & Heritage,Endangered,https://www.environment.nsw.gov.au/resources/a...,,obscured,...,Caladenia,concolor,,2022-06-10T07:36:31Z,Caladenia concolor,species,http://www.catalogueoflife.org/annual-checklis...,,,
2637,164342,913024,58320,6825,,NSW Office of Environment & Heritage,Vulnerable,https://www.environment.nsw.gov.au/threatenedS...,,,...,Symplocos,baeuerlenii,,2022-03-14T15:27:03Z,Symplocos baeuerlenii,species,,,,
7,167825,953250,702203,6825,,New South Wales Office of Environment and Heri...,VU,https://www.environment.nsw.gov.au/threatenedS...,Vulnerable,,...,Pultenaea,glabra,,2021-05-02T14:08:01Z,Pultenaea glabra,species,http://www.catalogueoflife.org/annual-checklis...,,,
836,155144,966856,708886,6825,16650,NSW Office of Environment & Heritage,endangered,https://www.environment.nsw.gov.au/resources/a...,,obscured,...,Pteris,platyzomopsis,,2019-11-20T05:25:13Z,Pteris platyzomopsis,species,https://eol.org/pages/47172990,,,


### 2. iNaturalist taxonomy

In [66]:
# Output files contain these fields
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# so we need to match species from the state lists to the inat taxa to get the taxon_id

import zipfile
url = "https://www.inaturalist.org/taxa/inaturalist-taxonomy.dwca.zip"
filename = url.split("/")[-1]

z=zipfile.ZipFile(sourcedir + filename)

with z.open('taxa.csv') as from_archive:
    inattaxa = pd.read_csv(from_archive,dtype=str)
z.close()
inattaxa.head(3)


,id,taxonID,identifier,parentNameUsageID,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references
0,1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/48460,Animalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-02T06:05:44Z,Animalia,kingdom,http://www.catalogueoflife.org/annual-checklis...
1,2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/1,Animalia,Chordata,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-23T00:40:18Z,Chordata,phylum,http://www.catalogueoflife.org/annual-checklis...
2,3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/355675,Animalia,Chordata,Aves,NaN,NaN,NaN,NaN,NaN,2022-12-27T07:33:16Z,Aves,class,http://www.catalogueoflife.org/annual-checklis...


### 3. State lists

Sensitive list: `geoprivacy` = `obscured`
overrides


In [67]:
sensitivelist = lf.download_ala_list("https://lists.ala.org.au/ws/speciesListItems/dr487?max=10000&includeKVP=true")
sensitivelist['scientificName'] = sensitivelist['scientificName'].str.replace('subsp. ', '', regex=False)
sensitivelist = sensitivelist.rename(columns={'tsprofileID':'bionetTaxonID'})
sensitivelist['bionet_geoprivacy'] = 'obscured'
sensitivelist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,bionet_geoprivacy
0,4040135,Callocephalon fimbriatum,Gang-gang Cockatoo,Callocephalon fimbriatum,https://biodiversity.org.au/afd/taxa/6c646af8-...,dr487,"[{'key': 'taxonRank', 'value': 'Species'}, {'k...",obscured
1,4040193,Ninox strenua,Powerful Owl,Ninox (Rhabdoglaux) strenua,https://biodiversity.org.au/afd/taxa/d1c5dee0-...,dr487,"[{'key': 'taxonRank', 'value': 'Species'}, {'k...",obscured
2,4040029,Tyto tenebricosa,Sooty Owl,Tyto tenebricosa,https://biodiversity.org.au/afd/taxa/645b287c-...,dr487,"[{'key': 'taxonRank', 'value': 'Species'}, {'k...",obscured
3,4040138,Ninox connivens,Barking Owl,Ninox (Hieracoglaux) connivens,https://biodiversity.org.au/afd/taxa/bd332ca4-...,dr487,"[{'key': 'taxonRank', 'value': 'Species'}, {'k...",obscured
4,4040203,Hoplocephalus bungaroides,Broad-headed Snake,Hoplocephalus bungaroides,https://biodiversity.org.au/afd/taxa/f579a6fc-...,dr487,"[{'key': 'taxonRank', 'value': 'Species'}, {'k...",obscured
...,...,...,...,...,...,...,...,...
192,4040083,Anthosachne kingiana subsp. kingiana,Phillip Island Wheat Grass,Anthosachne kingiana kingiana,https://id.biodiversity.org.au/taxon/apni/5129...,dr487,"[{'key': 'taxonRank', 'value': 'Subspecies'}, ...",obscured
193,4040198,Caladenia tensa,Erect Greencomb Spider Orchid,Caladenia tensa,https://id.biodiversity.org.au/taxon/apni/5139...,dr487,"[{'key': 'taxonRank', 'value': 'Species'}, {'k...",obscured
194,4040162,Caladenia atroclavia,Black-clubbed Spider-orchid,Caladenia atroclavia,https://id.biodiversity.org.au/taxon/apni/5139...,dr487,"[{'key': 'taxonRank', 'value': 'Species'}, {'k...",obscured
195,4040056,Prasophyllum sandrae,Majors Creek Leek Orchid,Prasophyllum sandrae,ALA_DR487_196,dr487,"[{'key': 'taxonRank', 'value': 'Species'}, {'k...",obscured


In [68]:
def gpcondition(x):
    if x=='Category 1':
        return "private"
    elif x== 'Category 2' or x=='Category 3':
        return "obscured"
    else:
        return 'open'

In [69]:
conservationlist = lf.download_ala_list("https://lists.ala.org.au/ws/speciesListItems/dr650?max=10000&includeKVP=true")
conservationlist = lf.kvp_to_columns(conservationlist)
conservationlist['scientificName'] = conservationlist['scientificName'].str.replace('subsp. ', '', regex=False)
conservationlist = conservationlist.rename(columns={'TSProfileID':'bionetTaxonID'})
conservationlist['bionet_geoprivacy'] = conservationlist['sensitivityClass'].apply(gpcondition)
conservationlist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,taxonRank,kingdom,class,...,sourceStatus,protectedInNSW,sensitivityClass,T S Profile I D,countryConservation,dcterms_modified,speciesID,taxonID,status,bionet_geoprivacy
0,4028582,Delma impar,Many-lined Delma,Delma impar,https://biodiversity.org.au/afd/taxa/51c700dd-...,dr650,"[{'key': 'taxonRank', 'value': 'Species'}, {'k...",Species,Animalia,Reptilia,...,Vulnerable,True,Not Sensitive,10211.0,Vulnerable,2013-12-20T15:06:07.13+11:00,1,1,Vulnerable,open
0,4028454,Callocephalon fimbriatum,Gang-gang Cockatoo,Callocephalon fimbriatum,https://biodiversity.org.au/afd/taxa/6c646af8-...,dr650,"[{'key': 'taxonRank', 'value': 'Species'}, {'k...",Species,Animalia,Aves,...,Vulnerable,True,Category 3,10975.0,Endangered,2011-11-11T11:23:09+11:00,6,6,Vulnerable,obscured
0,4028239,Cacophis harriettae,White-crowned Snake,Cacophis harriettae,https://biodiversity.org.au/afd/taxa/ace6534f-...,dr650,"[{'key': 'taxonRank', 'value': 'Species'}, {'k...",Species,Animalia,Reptilia,...,Vulnerable,True,Not Sensitive,10117.0,Not Listed,2015-05-28T10:14:29.083+10:00,31,31,Vulnerable,open
0,4028340,Litoria booroolongensis,Booroolong Frog,Litoria booroolongensis,https://biodiversity.org.au/afd/taxa/6377e038-...,dr650,"[{'key': 'taxonRank', 'value': 'Species'}, {'k...",Species,Animalia,Amphibia,...,Endangered,True,Not Sensitive,10484.0,Endangered,2011-11-11T11:23:05+11:00,33,33,Endangered,open
0,4028512,Anthochaera phrygia,Regent Honeyeater,Anthochaera (Xanthomyza) phrygia,https://biodiversity.org.au/afd/taxa/31869a0e-...,dr650,"[{'key': 'taxonRank', 'value': 'Species'}, {'k...",Species,Animalia,Aves,...,Critically Endangered,True,Not Sensitive,10841.0,Critically Endangered,2011-11-11T11:23:09+11:00,53,53,Critically Endangered,open
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,4028492,Muehlenbeckia sp. Mt Norman,Scrambling Lignum,Muehlenbeckia sp. Mt Norman,ALA_DR650_1014,dr650,"[{'key': 'taxonRank', 'value': 'Species'}, {'k...",Species,Plantae,Flora,...,Vulnerable,False,Not Sensitive,10547.0,Not Listed,2021-03-18T10:54:03.1+11:00,24272,24272,Vulnerable,open
0,4028995,Lobelia claviflora,None,Lobelia claviflora,ALA_DR650_1015,dr650,"[{'key': 'taxonRank', 'value': 'Species'}, {'k...",Species,Plantae,Flora,...,Critically Endangered,False,Not Sensitive,20375.0,Not Listed,2021-09-24T16:27:42.707+10:00,24282,24282,Critically Endangered,open
0,4029167,Fontainea sp. Coffs Harbour,None,Fontainea sp. Coffs Harbour,ALA_DR650_1016,dr650,"[{'key': 'taxonRank', 'value': 'Species'}, {'k...",Species,Plantae,Flora,...,Critically Endangered,False,Not Sensitive,20376.0,Not Listed,2021-10-15T10:15:24.41+11:00,24283,24283,Critically Endangered,open
0,4029112,Melichrus gibberagee,Narrow-leaf Melichrus,Melichrus gibberagee,ALA_DR650_1017,dr650,"[{'key': 'taxonRank', 'value': 'Species'}, {'k...",Species,Plantae,Flora,...,Endangered,False,Not Sensitive,10521.0,Endangered,2021-10-22T14:59:20.563+11:00,24284,24284,Endangered,open


In [73]:
# join them in a way that works for inat (eg sensitive list, geoprivacy = 'obscured'
statelist = pd.concat([sensitivelist[['scientificName', 'bionet_geoprivacy', 'lsid']],
                    conservationlist[['scientificName','status','bionet_geoprivacy', 'lsid']]]).drop_duplicates()
statelist

,scientificName,bionet_geoprivacy,lsid,status
0,Callocephalon fimbriatum,obscured,https://biodiversity.org.au/afd/taxa/6c646af8-...,NaN
1,Ninox (Rhabdoglaux) strenua,obscured,https://biodiversity.org.au/afd/taxa/d1c5dee0-...,NaN
2,Tyto tenebricosa,obscured,https://biodiversity.org.au/afd/taxa/645b287c-...,NaN
3,Ninox (Hieracoglaux) connivens,obscured,https://biodiversity.org.au/afd/taxa/bd332ca4-...,NaN
4,Hoplocephalus bungaroides,obscured,https://biodiversity.org.au/afd/taxa/f579a6fc-...,NaN
...,...,...,...,...
0,Muehlenbeckia sp. Mt Norman,open,ALA_DR650_1014,Vulnerable
0,Lobelia claviflora,open,ALA_DR650_1015,Critically Endangered
0,Fontainea sp. Coffs Harbour,open,ALA_DR650_1016,Critically Endangered
0,Melichrus gibberagee,open,ALA_DR650_1017,Endangered


In [75]:
# check for duplicates with conflicting information
statelist.groupby('scientificName').filter(lambda x: len(x) > 1)#.sort('size',ascending=False)
#df.groupby('hash').filter(lambda group: len(group) > 1).sort('size', ascending=False)

,scientificName,bionet_geoprivacy,lsid,status
0,Callocephalon fimbriatum,obscured,https://biodiversity.org.au/afd/taxa/6c646af8-...,NaN
1,Ninox (Rhabdoglaux) strenua,obscured,https://biodiversity.org.au/afd/taxa/d1c5dee0-...,NaN
2,Tyto tenebricosa,obscured,https://biodiversity.org.au/afd/taxa/645b287c-...,NaN
3,Ninox (Hieracoglaux) connivens,obscured,https://biodiversity.org.au/afd/taxa/bd332ca4-...,NaN
4,Hoplocephalus bungaroides,obscured,https://biodiversity.org.au/afd/taxa/f579a6fc-...,NaN
...,...,...,...,...
0,Amytornis (Amytornis) modestus inexpectatus,obscured,https://biodiversity.org.au/afd/taxa/71c46068-...,Extinct
0,Amytornis (Amytornis) modestus obscurior,obscured,https://biodiversity.org.au/afd/taxa/3e943ee9-...,Critically Endangered
0,Anthosachne kingiana kingiana,obscured,https://id.biodiversity.org.au/taxon/apni/5129...,Critically Endangered
0,Prasophyllum sandrae,obscured,ALA_DR487_196,Critically Endangered


### 4. Equivalent IUCN statuses

In [76]:
# iucn_statuses = {'Not Evaluated', 'Data Deficient', 'Least Concern', 'Near Threatened', 'Vulnerable', 'Endangered', 'Critically Endangered', 'Extinct in the Wild', 'Extinct'}
statelist.groupby(['status'])['status'].count()

status
Critically Endangered    114
Endangered               426
Extinct                   72
Vulnerable               406
Name: status, dtype: int64

In [77]:
iucnStatusMappings = {
    'Not Listed':'Not Evaluated',
}

### 5. Determine best place ID to use

In [78]:
inatstatuses.groupby(['place_id','place_name','place_display_name'])['place_id'].count()
# looks like 6825

place_id  place_name       place_display_name 
6744      Australia        Australia                5
6825      New South Wales  New South Wales, AU    161
Name: place_id, dtype: int64

## Merge iNaturalist statuses with State sensitive list on scientificName

1. Match - updates, even if the statuses are the same we'll update the links and values anyway
2. No match - statuses to be added (additions)
   1.1 No match and no taxnomy - search for synonyms
   1.2 No match
3. Merge the other direction to see if there are deletes?


In [80]:
# join to see which lists already have a status in inaturalist based on scientificName
mergedstatuses = statelist[['scientificName','status','bionet_geoprivacy', 'lsid']].merge(inatstatuses[['status_id','scientificName','taxon_id','user_id','description','iucn','authority','status','geoprivacy','place_id','place_display_name']],how="left",left_on='scientificName',right_on='scientificName',suffixes=(None,'_inat')).sort_values(['scientificName'])

mergedstatuses

,scientificName,status,bionet_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,status_inat,geoprivacy,place_id,place_display_name
728,Acacia acanthoclada,Endangered,open,https://id.biodiversity.org.au/node/apni/2898796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
476,Acacia acrionastes,Endangered,open,https://id.biodiversity.org.au/node/apni/2900521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1013,Acacia atrox,Endangered,obscured,https://id.biodiversity.org.au/node/apni/2894816,152295,898643,708886,,40,NSW Office of Environment & Heritage,endangered,obscured,6825,"New South Wales, AU"
141,Acacia atrox,NaN,obscured,https://id.biodiversity.org.au/node/apni/2894816,152295,898643,708886,,40,NSW Office of Environment & Heritage,endangered,obscured,6825,"New South Wales, AU"
559,Acacia ausfeldii,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2886531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Zieria parrisiae,NaN,obscured,https://id.biodiversity.org.au/node/apni/2919089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
910,Zieria prostrata,Endangered,open,https://id.biodiversity.org.au/node/apni/2915817,165911,897289,3669610,,40,NSW Office of Environment & Heritage,EN,,6825,"New South Wales, AU"
911,Zieria tuberculata,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2901823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
935,Zosterops lateralis tephropleurus,Vulnerable,open,https://biodiversity.org.au/afd/taxa/fdbd023e-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
# prepare the export fields, common to New template and Update template
# new statuses
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# updates
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username
mergedstatuses['new_authority'] = "New South Wales Office of Environment and Heritage"
mergedstatuses['new_description'] = "Listed as Threatened - refer to https://www.environment.nsw.gov.au/threatenedspeciesapp"
mergedstatuses['new_url'] = mergedstatuses['lsid']
mergedstatuses.rename(columns={'bionet_geoprivacy':'new_geoprivacy'},inplace=True)
mergedstatuses['new_place_id'] = '6825'  # NEW SOUTH WALES
mergedstatuses['new_username'] = 'peggydnew'
mergedstatuses['new_iucn_equivalent'] = mergedstatuses['status'].str.lower().str.strip().map(iucnStatusMappings).fillna('Vulnerable') # map to dictionary
mergedstatuses['new_status'] = mergedstatuses['status'].fillna('Threatened')
mergedstatuses

,scientificName,status,new_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,geoprivacy,place_id,place_display_name,new_authority,new_description,new_url,new_place_id,new_username,new_iucn_equivalent,new_status
728,Acacia acanthoclada,Endangered,open,https://id.biodiversity.org.au/node/apni/2898796,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,New South Wales Office of Environment and Heri...,Listed as Threatened - refer to https://www.en...,https://id.biodiversity.org.au/node/apni/2898796,6825,peggydnew,Vulnerable,Endangered
476,Acacia acrionastes,Endangered,open,https://id.biodiversity.org.au/node/apni/2900521,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,New South Wales Office of Environment and Heri...,Listed as Threatened - refer to https://www.en...,https://id.biodiversity.org.au/node/apni/2900521,6825,peggydnew,Vulnerable,Endangered
1013,Acacia atrox,Endangered,obscured,https://id.biodiversity.org.au/node/apni/2894816,152295,898643,708886,,40,NSW Office of Environment & Heritage,...,obscured,6825,"New South Wales, AU",New South Wales Office of Environment and Heri...,Listed as Threatened - refer to https://www.en...,https://id.biodiversity.org.au/node/apni/2894816,6825,peggydnew,Vulnerable,Endangered
141,Acacia atrox,NaN,obscured,https://id.biodiversity.org.au/node/apni/2894816,152295,898643,708886,,40,NSW Office of Environment & Heritage,...,obscured,6825,"New South Wales, AU",New South Wales Office of Environment and Heri...,Listed as Threatened - refer to https://www.en...,https://id.biodiversity.org.au/node/apni/2894816,6825,peggydnew,Vulnerable,Threatened
559,Acacia ausfeldii,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2886531,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,New South Wales Office of Environment and Heri...,Listed as Threatened - refer to https://www.en...,https://id.biodiversity.org.au/node/apni/2886531,6825,peggydnew,Vulnerable,Vulnerable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Zieria parrisiae,NaN,obscured,https://id.biodiversity.org.au/node/apni/2919089,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,New South Wales Office of Environment and Heri...,Listed as Threatened - refer to https://www.en...,https://id.biodiversity.org.au/node/apni/2919089,6825,peggydnew,Vulnerable,Threatened
910,Zieria prostrata,Endangered,open,https://id.biodiversity.org.au/node/apni/2915817,165911,897289,3669610,,40,NSW Office of Environment & Heritage,...,,6825,"New South Wales, AU",New South Wales Office of Environment and Heri...,Listed as Threatened - refer to https://www.en...,https://id.biodiversity.org.au/node/apni/2915817,6825,peggydnew,Vulnerable,Endangered
911,Zieria tuberculata,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2901823,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,New South Wales Office of Environment and Heri...,Listed as Threatened - refer to https://www.en...,https://id.biodiversity.org.au/node/apni/2901823,6825,peggydnew,Vulnerable,Vulnerable
935,Zosterops lateralis tephropleurus,Vulnerable,open,https://biodiversity.org.au/afd/taxa/fdbd023e-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,New South Wales Office of Environment and Heri...,Listed as Threatened - refer to https://www.en...,https://biodiversity.org.au/afd/taxa/fdbd023e-...,6825,peggydnew,Vulnerable,Vulnerable


## Updates

In [82]:
# updates
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
updates = pd.DataFrame(mergedstatuses[mergedstatuses['status_id'].notnull()])
updates.sort_values('scientificName')
updates['action'] = 'UPDATE'
#updates.loc[:,'action'] = 'UPDATE'
updates = updates[['action','scientificName','status_id','taxon_id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
updates.columns = updates.columns.str.replace("new_", "", regex=True)
updates = updates.rename(columns={'scientificName':'taxon_name',
                                  'status_id':'id'})
updates

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
1013,UPDATE,Acacia atrox,152295,898643,Endangered,Vulnerable,New South Wales Office of Environment and Heri...,https://id.biodiversity.org.au/node/apni/2894816,obscured,6825,peggydnew,Listed as Threatened - refer to https://www.en...
141,UPDATE,Acacia atrox,152295,898643,Threatened,Vulnerable,New South Wales Office of Environment and Heri...,https://id.biodiversity.org.au/node/apni/2894816,obscured,6825,peggydnew,Listed as Threatened - refer to https://www.en...
694,UPDATE,Acacia baueri aspera,159920,1111698,Vulnerable,Vulnerable,New South Wales Office of Environment and Heri...,https://id.biodiversity.org.au/node/apni/2892219,open,6825,peggydnew,Listed as Threatened - refer to https://www.en...
814,UPDATE,Acacia dangarensis,152286,775137,Critically Endangered,Vulnerable,New South Wales Office of Environment and Heri...,https://id.biodiversity.org.au/node/apni/2915433,obscured,6825,peggydnew,Listed as Threatened - refer to https://www.en...
97,UPDATE,Acacia dangarensis,152286,775137,Threatened,Vulnerable,New South Wales Office of Environment and Heri...,https://id.biodiversity.org.au/node/apni/2915433,obscured,6825,peggydnew,Listed as Threatened - refer to https://www.en...
...,...,...,...,...,...,...,...,...,...,...,...,...
898,UPDATE,Viola cleistogamoides,152321,566603,Endangered,Vulnerable,New South Wales Office of Environment and Heri...,https://id.biodiversity.org.au/node/apni/2892487,obscured,6825,peggydnew,Listed as Threatened - refer to https://www.en...
144,UPDATE,Wollemia nobilis,152312,49381,Threatened,Vulnerable,New South Wales Office of Environment and Heri...,https://id.biodiversity.org.au/node/apni/2899238,obscured,6825,peggydnew,Listed as Threatened - refer to https://www.en...
1018,UPDATE,Wollemia nobilis,152312,49381,Critically Endangered,Vulnerable,New South Wales Office of Environment and Heri...,https://id.biodiversity.org.au/node/apni/2899238,private,6825,peggydnew,Listed as Threatened - refer to https://www.en...
797,UPDATE,Zieria involucrata,266172,604220,Endangered,Vulnerable,New South Wales Office of Environment and Heri...,https://id.biodiversity.org.au/node/apni/2888006,open,6825,peggydnew,Listed as Threatened - refer to https://www.en...


## No status in iNaturalist via straight scientificName match
The NSW records that didn't match up to a status in iNaturalist

In [83]:
# to add: those that have no inaturalist status - 532!!
noinatstatus = mergedstatuses[mergedstatuses['status_id'].isnull()]
# try to match the taxon name to something in inaturalist
noinatstatus = noinatstatus.merge(inattaxa, how="left", left_on="scientificName",right_on="scientificName")
noinatstatus

,scientificName,status,new_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
0,Acacia acanthoclada,Endangered,open,https://id.biodiversity.org.au/node/apni/2898796,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,acanthoclada,NaN,2022-08-22T22:51:08Z,species,http://www.catalogueoflife.org/annual-checklis...
1,Acacia acrionastes,Endangered,open,https://id.biodiversity.org.au/node/apni/2900521,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,acrionastes,NaN,2022-04-06T22:12:50Z,species,http://powo.science.kew.org/
2,Acacia ausfeldii,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2886531,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,ausfeldii,NaN,2022-06-07T09:01:00Z,species,http://www.catalogueoflife.org/annual-checklis...
3,Acacia bakeri,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2902312,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,bakeri,NaN,2022-04-07T01:35:47Z,species,http://www.catalogueoflife.org/annual-checklis...
4,Acacia bynoeana,Endangered,open,https://id.biodiversity.org.au/node/apni/2897229,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,bynoeana,NaN,2022-04-07T01:36:15Z,species,http://www.catalogueoflife.org/annual-checklis...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,Zieria parrisiae,Critically Endangered,obscured,https://id.biodiversity.org.au/node/apni/2919089,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,parrisiae,NaN,2020-02-19T19:35:12Z,species,https://eol.org/pages/49431665
989,Zieria parrisiae,NaN,obscured,https://id.biodiversity.org.au/node/apni/2919089,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,parrisiae,NaN,2020-02-19T19:35:12Z,species,https://eol.org/pages/49431665
990,Zieria tuberculata,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2901823,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,tuberculata,NaN,2021-07-28T04:47:52Z,species,https://eol.org/pages/49431639
991,Zosterops lateralis tephropleurus,Vulnerable,open,https://biodiversity.org.au/afd/taxa/fdbd023e-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,lateralis,tephropleurus,2020-01-16T00:05:01Z,subspecies,NaN


In [84]:
additions = pd.DataFrame(noinatstatus[noinatstatus['id'].notna()])
additions

,scientificName,status,new_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
0,Acacia acanthoclada,Endangered,open,https://id.biodiversity.org.au/node/apni/2898796,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,acanthoclada,NaN,2022-08-22T22:51:08Z,species,http://www.catalogueoflife.org/annual-checklis...
1,Acacia acrionastes,Endangered,open,https://id.biodiversity.org.au/node/apni/2900521,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,acrionastes,NaN,2022-04-06T22:12:50Z,species,http://powo.science.kew.org/
2,Acacia ausfeldii,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2886531,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,ausfeldii,NaN,2022-06-07T09:01:00Z,species,http://www.catalogueoflife.org/annual-checklis...
3,Acacia bakeri,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2902312,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,bakeri,NaN,2022-04-07T01:35:47Z,species,http://www.catalogueoflife.org/annual-checklis...
4,Acacia bynoeana,Endangered,open,https://id.biodiversity.org.au/node/apni/2897229,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,bynoeana,NaN,2022-04-07T01:36:15Z,species,http://www.catalogueoflife.org/annual-checklis...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,Zieria parrisiae,Critically Endangered,obscured,https://id.biodiversity.org.au/node/apni/2919089,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,parrisiae,NaN,2020-02-19T19:35:12Z,species,https://eol.org/pages/49431665
989,Zieria parrisiae,NaN,obscured,https://id.biodiversity.org.au/node/apni/2919089,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,parrisiae,NaN,2020-02-19T19:35:12Z,species,https://eol.org/pages/49431665
990,Zieria tuberculata,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2901823,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Sapindales,Rutaceae,Zieria,tuberculata,NaN,2021-07-28T04:47:52Z,species,https://eol.org/pages/49431639
991,Zosterops lateralis tephropleurus,Vulnerable,open,https://biodiversity.org.au/afd/taxa/fdbd023e-...,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,lateralis,tephropleurus,2020-01-16T00:05:01Z,subspecies,NaN


In [85]:
# there's no status but there is a matching inat taxon (id is the taxon id)
additions = pd.DataFrame(noinatstatus[noinatstatus['id'].notna()])
additions.sort_values(['scientificName'])
additions['action'] = 'ADD'
additions = additions[['action','scientificName','status_id','id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
additions.columns = additions.columns.str.replace("new_", "", regex=True)
additions = additions.rename(columns={'scientificName':'taxon_name',
                                      'id':'taxon_id',
                                  'status_id':'id'})
additions

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
0,ADD,Acacia acanthoclada,NaN,787701,Endangered,Vulnerable,New South Wales Office of Environment and Heri...,https://id.biodiversity.org.au/node/apni/2898796,open,6825,peggydnew,Listed as Threatened - refer to https://www.en...
1,ADD,Acacia acrionastes,NaN,898576,Endangered,Vulnerable,New South Wales Office of Environment and Heri...,https://id.biodiversity.org.au/node/apni/2900521,open,6825,peggydnew,Listed as Threatened - refer to https://www.en...
2,ADD,Acacia ausfeldii,NaN,139906,Vulnerable,Vulnerable,New South Wales Office of Environment and Heri...,https://id.biodiversity.org.au/node/apni/2886531,open,6825,peggydnew,Listed as Threatened - refer to https://www.en...
3,ADD,Acacia bakeri,NaN,524410,Vulnerable,Vulnerable,New South Wales Office of Environment and Heri...,https://id.biodiversity.org.au/node/apni/2902312,open,6825,peggydnew,Listed as Threatened - refer to https://www.en...
4,ADD,Acacia bynoeana,NaN,543213,Endangered,Vulnerable,New South Wales Office of Environment and Heri...,https://id.biodiversity.org.au/node/apni/2897229,open,6825,peggydnew,Listed as Threatened - refer to https://www.en...
...,...,...,...,...,...,...,...,...,...,...,...,...
988,ADD,Zieria parrisiae,NaN,965722,Critically Endangered,Vulnerable,New South Wales Office of Environment and Heri...,https://id.biodiversity.org.au/node/apni/2919089,obscured,6825,peggydnew,Listed as Threatened - refer to https://www.en...
989,ADD,Zieria parrisiae,NaN,965722,Threatened,Vulnerable,New South Wales Office of Environment and Heri...,https://id.biodiversity.org.au/node/apni/2919089,obscured,6825,peggydnew,Listed as Threatened - refer to https://www.en...
990,ADD,Zieria tuberculata,NaN,1247618,Vulnerable,Vulnerable,New South Wales Office of Environment and Heri...,https://id.biodiversity.org.au/node/apni/2901823,open,6825,peggydnew,Listed as Threatened - refer to https://www.en...
991,ADD,Zosterops lateralis tephropleurus,NaN,980046,Vulnerable,Vulnerable,New South Wales Office of Environment and Heri...,https://biodiversity.org.au/afd/taxa/fdbd023e-...,open,6825,peggydnew,Listed as Threatened - refer to https://www.en...


In [86]:
# write these to the file
pd.concat([updates,additions]).to_csv(sourcedir + "nsw.csv", index=False)

In [87]:
# what didnt match to a taxon?
unknownToInat = noinatstatus[noinatstatus['id'].isna()]
unknownToInat

,scientificName,status,new_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
23,Acacia terminalis Eastern Sydney,Endangered,open,ALA_DR650_1011,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,Amperea xiphoclada var. pedicellata,Extinct,open,https://id.biodiversity.org.au/node/apni/2910887,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,Amytornis (Amytornis) modestus inexpectatus,NaN,obscured,https://biodiversity.org.au/afd/taxa/71c46068-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,Amytornis (Amytornis) modestus inexpectatus,Extinct,obscured,https://biodiversity.org.au/afd/taxa/71c46068-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,Amytornis (Amytornis) modestus obscurior,NaN,obscured,https://biodiversity.org.au/afd/taxa/3e943ee9-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,Velleia perfoliata,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2920937,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
960,Wollumbinia belli,Endangered,open,https://biodiversity.org.au/afd/taxa/1a85a82f-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
961,Wollumbinia georgesi,Critically Endangered,obscured,https://biodiversity.org.au/afd/taxa/4f95257b-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
962,Wollumbinia georgesi,NaN,obscured,https://biodiversity.org.au/afd/taxa/4f95257b-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### are there any that need to be removed?
nsw sensitive list count: 198
nsw inat statuses count: 166

updates to inat status: 150
additional inat status: 732
nsw statuses we can't find a taxon match for in iNaturalist: 149
total: 1395 (explainable via the various genus/section entries that we matched to in the taxonomy)

inat statuses left over: 166-150=16 that may need checking

In [88]:
# inat statuses that aren't in added or updated
inatstatuses[~inatstatuses['taxon_id'].isin(updates['taxon_id'])]


,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
477,264941,1061113,3669610,6825,,New South Wales Biodiversity Conservation Act ...,EX,https://bie.ala.org.au/species/https://id.biod...,Presumed Extinct,open,...,Leuzea,australis,,2022-06-11T11:38:00Z,Leuzea australis,species,,,,
681,164757,1098159,58320,6825,,NSW Office of Environment & Heritage,Vulnerable,https://www.inaturalist.org/taxa/1098159/edit,,,...,Melaleuca,linearifolia,,2022-09-05T10:35:34Z,Melaleuca linearifolia,species,http://www.plantsoftheworldonline.org/,,,
1147,152307,116844,708886,6825,16650,NSW Office of Environment & Heritage,critically endangered,https://www.environment.nsw.gov.au/resources/a...,,obscured,...,Calyptorhynchus,banksii,,2019-11-23T01:11:55Z,Calyptorhynchus banksii,species,http://www.birdlife.org/datazone/speciesfactsh...,,,
2800,152257,116846,708886,6825,16650,NSW Office of Environment & Heritage,Vulnerable,https://www.environment.nsw.gov.au/resources/a...,,obscured,...,Calyptorhynchus,lathami,,2022-12-11T15:26:46Z,Calyptorhynchus lathami,species,http://www.birdlife.org/datazone/speciesfactsh...,,,
736,166035,1194478,3669610,6825,,NSW Office of Environment & Heritage,EN,https://www.environment.nsw.gov.au/threateneds...,,,...,Vincetoxicum,woollsii,,2021-02-11T14:40:16Z,Vincetoxicum woollsii,species,http://www.plantsoftheworldonline.org/taxon/ur...,,,
737,166036,1194478,3669610,6744,,Australian Government,EN,https://www.environment.nsw.gov.au/threateneds...,,,...,Vincetoxicum,woollsii,,2021-02-11T14:40:16Z,Vincetoxicum woollsii,species,http://www.plantsoftheworldonline.org/taxon/ur...,,,
51,167249,1227491,3669610,6825,,NSW Office of Environment & Heritage,EN,https://www.environment.nsw.gov.au/threateneds...,,,...,Phyllodes,imperialis,smithersi,2021-03-23T03:42:06Z,Phyllodes imperialis smithersi,subspecies,https://biodiversity.org.au/afd/taxa/Phyllodes...,,,
397,264070,1255425,3669610,6825,,NSW Office of Environment & Heritage,CR,https://www.environment.nsw.gov.au/threateneds...,,open,...,Melaleuca,megalongensis,,2022-09-07T09:45:08Z,Melaleuca megalongensis,species,http://www.catalogueoflife.org/annual-checklis...,,,
1181,152326,13480,708886,6825,16650,NSW Office of Environment & Heritage,Endangered,https://www.environment.nsw.gov.au/resources/a...,,obscured,...,Dasyornis,brachypterus,,2022-06-10T12:33:53Z,Dasyornis brachypterus,species,http://www.birdlife.org/datazone/speciesfactsh...,,,
1162,152325,146300,708886,6825,16650,NSW Office of Environment & Heritage,critically endangered,https://www.environment.nsw.gov.au/resources/a...,,obscured,...,Myuchelys,georgesi,,2019-11-23T07:02:11Z,Myuchelys georgesi,species,http://www.reptile-database.org,,,
